In [1]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
import pandas as pd
import time

In [2]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [7]:
# collect the latest News Title and Paragraph Text. Assign the text to variables that you can reference later.
mars_url = 'https://mars.nasa.gov/news/'
browser.visit(mars_url)
newshtml = browser.html
newsoup = bs(newshtml, 'html.parser')
time.sleep(3)
news_title = newsoup.find('div', class_='content_title')
news_p = newsoup.find('div', class_='article_teaser_body')
print(news_title.text.strip())
print(news_p.text.strip())

Curiosity Tastes First Sample in 'Clay-Bearing Unit'
This new region on Mars might reveal more about the role of water on Mount Sharp.


In [9]:
# find the image url for the current Featured Mars Image and assign the url string to a variable
jpl_url_search = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
jpl_url = 'https://www.jpl.nasa.gov'
jplhtml = browser.html
jplsoup = bs(jplhtml, 'html.parser')
browser.visit(jpl_url_search)
time.sleep(1)
browser.click_link_by_partial_text('FULL IMAGE')
time.sleep(1)
# browser.find_by_css('a.fancybox-expand').click()
# time.sleep(1)
img_path = jplsoup.find('img', class_='fancybox-image')['src']
featured_image_url = jpl_url + img_path
print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA19637_ip.jpg


In [11]:
# scrape the latest Mars weather tweet
# twitter log in window pops up during initial run, should work second time running
marsweatherurl = 'https://twitter.com/marswxreport?lang=en'
mars_html = browser.html
mars_soup = bs(mars_html, 'html.parser')
browser.visit(marsweatherurl)
time.sleep(2)
mars_weather = mars_soup.find('p', class_='TweetTextSize')
print(mars_weather.text.strip())

InSight sol 132 (2019-04-10) low -97.7ºC (-143.8ºF) high -16.3ºC (2.7ºF)
winds from the SW at 4.3 m/s (9.5 mph) gusting to 12.2 m/s (27.3 mph)
pressure at 7.30 hPapic.twitter.com/lW713h6MMh


In [12]:
# use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc
mars_facts_url = 'https://space-facts.com/mars/'
facts_html = browser.html
facts_soup = bs(facts_html, 'html.parser')
browser.visit(mars_facts_url)
table = pd.read_html(mars_facts_url)
df = table[0]
df = df.rename(columns={0: 'description', 1: 'value'})

In [13]:
df

,description,value
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.42 x 10^23 kg (10.7% Earth)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.52 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-153 to 20 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [14]:
# obtain high resolution images for each of Mar's hemispheres
hemisph_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
hlist = ['Cerberus Hemisphere', 'Schiaparelli Hemisphere', 'Syrtis Major Hemisphere', 'Valles Marineris Hemisphere']
hemisph_img_urls = []
for h in hlist:
    img_dict = {}
    browser.visit(hemisph_url)
    browser.click_link_by_partial_text(h)
    time.sleep(1)
    browser.find_link_by_text('Sample').first.click()
    
    browser.windows.current = browser.windows[-1]
    hemisph_soup = bs(browser.html, 'html.parser')
    img_url = hemisph_soup.find('img')['src']
    img_dict = {'title': h, 'img_url': img_url}
    hemisph_img_urls.append(img_dict)   
    browser.windows[-1].close()
    
    time.sleep(1)

In [15]:
hemisph_img_urls

[{'title': 'Cerberus Hemisphere',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]